In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from torch.utils.data import Dataset

In [2]:
# Step 1: Load and Preprocess Data
fake = pd.read_csv("C:/Users/OMEN/Desktop/FakeNews_Best_Model/archive/Fake.csv")
true = pd.read_csv("C:/Users/OMEN/Desktop/FakeNews_Best_Model/archive/True.csv")

In [3]:
# Drop irrelevant columns
fake.drop(['subject', 'date'], axis=1, inplace=True)
true.drop(['subject', 'date'], axis=1, inplace=True)
# Assign labels
fake['label'] = 1
true['label'] = 0

In [4]:
# Combine and shuffle
data = pd.concat([fake, true], axis=0).sample(frac=1).reset_index(drop=True)
data.fillna(' ', inplace=True)

In [5]:
# Extract raw text and labels
texts = data['text'].tolist()
labels = data['label'].tolist()

In [6]:
# 🔻 Optional: use small subset for fast training
texts = texts[:2000]
labels = labels[:2000]

In [7]:
# Step 2: Train/Test Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

In [8]:
# Step 2: Load small BERT model
model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Step 3: Tokenize (shorter sequence for speed)
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} | {
            "labels": torch.tensor(self.labels[idx])
        }
    def __len__(self):
        return len(self.labels)

train_dataset = FakeNewsDataset(train_encodings, train_labels)
val_dataset = FakeNewsDataset(val_encodings, val_labels)

In [12]:
# Step 4: Training settings
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

c:\Users\OMEN\anaconda3\envs\BERT\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
# Step 5: Train and evaluate
trainer.train()
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

Epoch,Training Loss,Validation Loss
1,0.392600,0.338757
2,0.245400,0.215380


Evaluation Results: {'eval_loss': 0.2153802514076233, 'eval_runtime': 0.147, 'eval_samples_per_second': 2721.846, 'eval_steps_per_second': 170.115, 'epoch': 2.0}
